In [8]:
# !pip install transformers #Use this command to install transformers in colab
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime
import time
from transformers import pipeline

C:\Users\ysj30\anaconda3\envs\py38torch\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [9]:
print("Enter 1 -> for 125M, 2 -> 1.3B and 3 -> for 2.7B")

Enter 1 -> for 125M, 2 -> 1.3B and 3 -> for 2.7B


In [15]:
model_no = int(input("Enter 1 -> for 125M, 2 -> 1.3B and 3 -> for 2.7B"))

Enter 1 -> for 125M, 2 -> 1.3B and 3 -> for 2.7B2


In [16]:
model_no

2

In [17]:
if model_no == 1:
    model = pipeline('text-generation',model='EleutherAI/gpt-neo-125M')
elif model_no == 2:
    model = pipeline('text-generation',model='EleutherAI/gpt-neo-1.3B')
elif model_no == 3:
    model = pipeline('text-generation',model='EleutherAI/gpt-neo-2.7B')
else:
    model = 'No Model'

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 67108864 bytes.

In [18]:
#Internal Functions
cols = ['id','model_name','iteration','temperature','input_length','problem_statement','abstract_reformulation','cumulative_time_in_seconds']
def generate_blank_df(cols): 
    df = pd.DataFrame(columns = cols)
    return df

def get_current_date():
    now = datetime.now()
    dt_string = now.strftime("%Y%m%d")
    date_string = dt_string
    return date_string



In [19]:
date = get_current_date()

In [42]:
#Main Function
#number of problem statements - Number of input problem statements - required for setting limit of result and parsing the output
def generate_output(model,max_iterations,prompt,number_of_problem_statements,temp,model_name_text,want_to_save_1or0):
    start = time.time()
    date = get_current_date()
    df = generate_blank_df(cols)
    rand_id = ''.join([random.choice(string.ascii_letters
            + string.digits) for n in range(7)])
    iteration_no = 1
    columns = list(df)
    file_name = date+'_'+model_name_text+'_'+rand_id+'_'+str(max_iterations)+'_'+str(number_of_problem_statements)+'.csv'
    data = []
    while temp < 1.0:
        iteration_no = 1        
        while iteration_no <= max_iterations:
            cumulative_time=round((time.time()-start))
            res_model = model(prompt,max_length = 32*number_of_problem_statements,do_sample = True,temperature = temp) #Model Generation
            
            #Parsing
            problem_statement = res_model[0]['generated_text'].split('\n\n')[number_of_problem_statements+1].split('\n')[0].split(':')[-1].strip()
            abs_reformulation = res_model[0]['generated_text'].split('\n\n')[number_of_problem_statements+1].split('\n')[-1].split(':')[-1].strip()
            values_for_df = [rand_id,model_name_text,iteration_no,temp,number_of_problem_statements,problem_statement,abs_reformulation,cumulative_time]
            zipped = zip(columns,values_for_df)
            dict1 = dict(zipped)
            data.append(dict1)
            iteration_no += 1
            print(iteration_no-1,problem_statement,'->',abs_reformulation,cumulative_time,temp)
            print('----------------------------------------')
        
            df = df.append(data,ignore_index = True)
        temp = round((temp+0.1),1)
        
    df = df.drop_duplicates()
    
    #Saving to csv (Internal Save)
    if(want_to_save_1or0==1):
        df.to_csv(f'../Csv/{file_name}',index=False)
    else:
        print('returning_df')
    
    elapsed_time_lc=(time.time()-start)
    print(f'Total time taken is {round(elapsed_time_lc)}')
    
    return df
    

In [33]:
#Getting Input from a text File
with open('prompt.txt', 'r') as file:
    prompt = file.read()

In [34]:
# prompt

"Given a problem statement, generate an abstracted reformulation of that problem statement\n\nProblem statement: Shield baby from the sun\nAbstracted reformulation: Protect vulnerable object from radiation\n\nProblem statement: Prevent feet from getting wet\nAbstracted reformulation: Stop object from getting into contact with unwanted substances\n\nProblem statement: Hold soap bars of multiple sizes\nAbstracted reformulation: Contain objects of different volumes\n\nProblem statement: Enable a broom to clean in hard to reach places\nAbstracted reformulation: Extend the physical reach of a tool\n\nProblem statement: drill into teeth quietly\nAbstracted reformulation: create a hole in an object without causing an unwanted noise\n\nProblem statement: make house out of cooked spaghetti\nAbstracted reformulation: create strong, rigid structure out of soft materials\n\nProblem statement: make house out of cooked spaghetti\nAbstracted reformulation: create structure from elastic elements\n\nPr

In [35]:
user_input = "identify points of failure in a nuclear power plant procedure"

In [36]:
#Appending to prompt
prompt = prompt + user_input

In [38]:
# prompt

"Given a problem statement, generate an abstracted reformulation of that problem statement\n\nProblem statement: Shield baby from the sun\nAbstracted reformulation: Protect vulnerable object from radiation\n\nProblem statement: Prevent feet from getting wet\nAbstracted reformulation: Stop object from getting into contact with unwanted substances\n\nProblem statement: Hold soap bars of multiple sizes\nAbstracted reformulation: Contain objects of different volumes\n\nProblem statement: Enable a broom to clean in hard to reach places\nAbstracted reformulation: Extend the physical reach of a tool\n\nProblem statement: drill into teeth quietly\nAbstracted reformulation: create a hole in an object without causing an unwanted noise\n\nProblem statement: make house out of cooked spaghetti\nAbstracted reformulation: create strong, rigid structure out of soft materials\n\nProblem statement: make house out of cooked spaghetti\nAbstracted reformulation: create structure from elastic elements\n\nPr

In [40]:
model_name_text = 'gpt_neo_125M'

In [41]:
output = generate_output(model,10,prompt,20,0.9,model_name_text,1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1 identify points of failure in a nuclear power plant procedure -> prevent child from getting into a room with the same object 0 0.9
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2 identify points of failure in a nuclear power plant procedure -> identify points of failure in a nuclear power plant procedure 11 0.9
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

7 identify points of failure in a nuclear power plant procedure -> Prevent object from getting into contact with unwanted substances 59 0.9
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


8 identify points of failure in a nuclear power plant procedure -> Protect object from the penetration of foreign objects 70 0.9
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


9 identify points of failure in a nuclear power plant procedure -> identify failure points 80 0.9
----------------------------------------
10 identify points of failure in a nuclear power plant procedure -> Reduce energy consumption in devices 90 0.9
----------------------------------------
Total time taken is 100


In [43]:
output.to_csv(f'gpu_gpt_neo_27b_{user_input}.csv')